In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from dateutil.parser import parse #https://stackoverflow.com/questions/40121822/extracting-year-from-string-in-python
pd.set_option("display.max_rows", 999) # To control the number of rows to be display while working with the dataframe

data_list = []
with open('../data_electric_consumption/CNE_SD_DN_electric_compsution_kWh_reordered-csv.csv', 'r', encoding="ISO-8859-1") as f: #An error raises if this encoding is not specify
    for line in f.readlines():
        # print(line)
        new_line = line.replace(',,,,,,,,,,,,', '') #To better display the output... not needed in the end, should be dropped
        new_line = new_line.replace('"', '')
        # print(new_line)
        if new_line.startswith('Cantidad de'): #grepping this line in order to extract the year
            # print(new_line)
            yy = parse(new_line, fuzzy=True).year
            # print(yy)
        elif new_line.startswith('Distrito Nacional') or new_line.startswith('Santo Domingo') or new_line.startswith('Total'):
            # print(new_line)
            data_line = new_line.split(',')
            # print(data_line)
# #             # data_line.pop(0)
            data_line[-1] = data_line[-1].replace('\n', '')
            # print(data_line)
            PROV, ENE, FEB, MAR, ABR, MAY, JUN, JUL, AGO, SEP, OCT, NOV, DIC, TOTAL = data_line #Unpacking the list into its corresponding fields
            data_list.append([yy, PROV, ENE, FEB, MAR, ABR, MAY, JUN, JUL, AGO, SEP, OCT, NOV, DIC, TOTAL])
            
df = pd.DataFrame(data_list, columns = [
                                   'YEAR', 'PROV', 'ENE', 'FEB', 'MAR', 'ABR', 
                                   'MAY', 'JUN',  'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC', 'TOTAL'
                                       ] )

df.head()         

,YEAR,PROV,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGO,SEP,OCT,NOV,DIC,TOTAL
0,2000,Distrito Nacional,141323226,141200749,125889655,134927663,129622120,140590395,150540064,137392686,145286836,151020852,160595431,160478728,1718868405
1,2000,Santo Domingo,85123461,83637168,81952893,87743785,84288681,90679811,92632303,92208760,96881962,98747138,100382760,107330874,1101609596
2,2000,Total,226446687,224837917,207842548,222671448,213910801,231270206,243172367,229601446,242168798,249767990,260978191,267809602,2820478001
3,2001,Distrito Nacional,185685464,139656845,164992430,167098181,171020658,183743349,182414399,181330638,189660145,180295162,186056459,171246344,2103200074
4,2001,Santo Domingo,114558645,103326399,113194624,113471297,112293665,118442640,116003033,114391758,121337554,117993563,122879196,120235549,1388127923


In [2]:
df2 = pd.DataFrame(columns=['YEAR', 'MONTH', 'CONSUMPTION'])
data = []
months = ['ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN',  'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC']
grp_df = df.groupby(['YEAR'])
for year in range(2000, 2022, 1):
    # print(year)
    yr = grp_df.get_group(year) #yr is a dataframe containing the data for a single year like in the original file
    # print(yr)
    for month in months:
        # print(yr[month])
        for c,p in zip(yr[month], yr['PROV']):
            data.append([year, month, p, c])

df2 = pd.DataFrame(data, columns = [
                                   'YEAR', 'MONTH', 'PROV', 'ELECT_CONSUMT(_kW_h)'
                                 ] )


df2['MONTH_NUM'] = df2['MONTH']
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('ENE', '1')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('FEB', '2')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('MAR', '3')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('ABR', '4')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('MAY', '5')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('JUN', '6')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('JUL', '7')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('AGO', '8')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('SEP', '9')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('OCT', '10')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('NOV', '11')
df2['MONTH_NUM'] = df2['MONTH_NUM'].replace('DIC', '12')

df2.reset_index()
df2.tail()

,YEAR,MONTH,PROV,ELECT_CONSUMT(_kW_h),MONTH_NUM
787,2021,NOV,Santo Domingo,247227002,11
788,2021,NOV,Total,485633752,11
789,2021,DIC,Distrito Nacional,225349481,12
790,2021,DIC,Santo Domingo,199038721,12
791,2021,DIC,Total,424388201,12


In [3]:
df2.drop(df2.tail(12).index, inplace = True)
df2['DAY']  = '1'
df2['MONTH'] = df2['MONTH_NUM']
df2["DATE"] = pd.to_datetime(df2[["YEAR", "MONTH", "DAY"]], errors="coerce")
df2 = df2[["DATE", "PROV", "ELECT_CONSUMT(_kW_h)"]]
df2.tail()

,DATE,PROV,ELECT_CONSUMT(_kW_h)
775,2021-07-01,Santo Domingo,223057273
776,2021-07-01,Total,448989723
777,2021-08-01,Distrito Nacional,232417163
778,2021-08-01,Santo Domingo,213744399
779,2021-08-01,Total,446161562


In [4]:
# df2_dropped_last_n = df2.iloc[:-12]
# df2_dropped_last_n.tail()

In [5]:
df2['ELECT_CONSUMT(_kW_h)']      = pd.to_numeric(df2['ELECT_CONSUMT(_kW_h)'])
# df2 = df2.replace(" -   ", np.nan)

df2.set_index('DATE', inplace=True)
# df2.loc[df2['PROV'] == 'Distrito Nacional'].plot(figsize=(20, 10))

In [6]:
# df2.loc[df2['PROV'] == 'Santo Domingo'].plot(figsize=(20, 10))

In [7]:
# df2.loc[df2['PROV'] == 'Total'].plot(figsize=(20, 10))

In [8]:
df2.loc[df2['PROV'] == 'Distrito Nacional']['ELECT_CONSUMT(_kW_h)'].head()#.describe()

DATE
2000-01-01    141323226
2000-02-01    141200749
2000-03-01    125889655
2000-04-01    134927663
2000-05-01    129622120
Name: ELECT_CONSUMT(_kW_h), dtype: int64

In [9]:
df2['EC_DN'] = df2.loc[df2['PROV'] == 'Distrito Nacional']['ELECT_CONSUMT(_kW_h)']
df2['EC_SD'] = df2.loc[df2['PROV'] == 'Santo Domingo']['ELECT_CONSUMT(_kW_h)']
df2['EC_GSD'] = df2.loc[df2['PROV'] == 'Total']['ELECT_CONSUMT(_kW_h)']
df2 = df2[['EC_DN', 'EC_SD', 'EC_GSD']]
df2.drop_duplicates(inplace = True)
df2.head(20)

,EC_DN,EC_SD,EC_GSD
DATE,,,
2000-01-01,141323226,85123461,226446687
2000-02-01,141200749,83637168,224837917
2000-03-01,125889655,81952893,207842548
2000-04-01,134927663,87743785,222671448
2000-05-01,129622120,84288681,213910801
2000-06-01,140590395,90679811,231270206
2000-07-01,150540064,92632303,243172367
2000-08-01,137392686,92208760,229601446
2000-09-01,145286836,96881962,242168798


In [10]:
writer = pd.ExcelWriter('../time_series_cne_data.xlsx')
df2.to_excel(writer)
writer.save()